In [1]:
import torch, pickle, argparse
import numpy as np, pytorch_lightning as pl, matplotlib.pyplot as plt, eagerpy as ep
from models import ConvNet
from data_loader import load_train_data, load_test_data
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD

In [2]:
filename = 'first_tests.pickle'

with open(filename, 'rb') as file:
    loaded = pickle.load(file)
    
data = np.array(loaded[0][0]).reshape(3,6,3,2)
hparams = argparse.Namespace(**data[2,-1,1,1])

In [3]:
TEST_PATH = "s2_mnist.gz"
TRAIN_PATH = "s2_mnist_train_dwr_600000.gz"

train_data = load_train_data(TRAIN_PATH)
test_data = load_test_data(TEST_PATH)

In [4]:
model = ConvNet(hparams, train_data, test_data).eval()
trainer = pl.Trainer(gpus=1, logger=False)
best_model = torch.load('checkpoints/epoch=17_v6.ckpt')
model.load_state_dict(best_model['state_dict'])
model.eval()
# trainer.test(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


ConvNet(
  (loss_function): CrossEntropyLoss()
  (conv): Sequential(
    (0): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(12, 15, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(15, 16, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(16, 85, kernel_size=(7, 7), stride=(2, 2))
    (7): ReLU()
    (8): Conv2d(85, 141, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
    (10): Conv2d(141, 191, kernel_size=(3, 3), stride=(1, 1))
    (11): ReLU()
    (12): Conv2d(191, 1100, kernel_size=(3, 3), stride=(2, 2))
    (13): ReLU()
  )
  (dense): Sequential(
    (0): BatchNorm1d(1100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=1100, out_features=64, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(32, eps=1e-05, 

In [5]:
fmodel = PyTorchModel(model, bounds=(0, 255))
model = model.cpu()

In [6]:
images = test_data[:20][0]#.cuda()
labels = test_data[:20][1]#.cuda()
images = ep.astensor(images)
labels = ep.astensor(labels)

In [7]:
clean_acc = accuracy(fmodel, images, labels)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

clean accuracy:  100.0 %


In [8]:
attack = LinfPGD()
epsilons = np.array([
    0.0,
    0.0002,
    0.0005,
    0.0008,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.01,
    0.1,
    0.3,
    0.5,
    1,
])*255
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)

In [9]:
robust_accuracy = 1 - success.float32().mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 100.0 %
  Linf norm ≤ 0.051000000000000004: 100.0 %
  Linf norm ≤ 0.1275: 100.0 %
  Linf norm ≤ 0.20400000000000001: 100.0 %
  Linf norm ≤ 0.255 : 95.0 %
  Linf norm ≤ 0.3825: 80.0 %
  Linf norm ≤ 0.51  : 75.0 %
  Linf norm ≤ 0.765 : 65.0 %
  Linf norm ≤ 2.5500000000000003: 10.0 %
  Linf norm ≤ 25.5  :  0.0 %
  Linf norm ≤ 76.5  :  0.0 %
  Linf norm ≤ 127.5 :  0.0 %
  Linf norm ≤ 255.0 :  0.0 %


In [10]:
# i = 0
# print(labels.raw[i].item())
# plt.imshow(images.raw[i,0], cmap = 'gray')
# plt.show()

In [11]:
# model(images[0].raw.unsqueeze(0))

In [12]:
# labels[0].raw

In [13]:
# loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
# loss_fn(model(images[0].raw.unsqueeze(0)), labels[0].raw.unsqueeze(0))

In [15]:
# torch.amin(images[0].raw)